<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



# Curso *Ingeniería de Características*

### Descargando datos


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/integracion/python/descarga_datos.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a>

</center>

# 1. Descargando datos a la fuerza bruta

Vamos a ver primero como ir descargando datos y luego como lidiar con diferentes formatos. Es muy importante que, si los datos los vamos a cargar por única vez, descargar el conjunto de datos, tal como se encuentran, esto es `raw data`.

Vamos primero cargando las bibliotecas necesarias:

In [ ]:

import os  # Para manejo de archivos y directorios
import urllib.request # Una forma estandard de descargar datos
# import requests # Otra forma no de las librerías de uso comun

import datetime # Fecha de descarga
import pandas as pd # Solo para ver el archivo descargado
import zipfile # Descompresión de archivos

Es importante saber en donde nos encontramos y crear los subdirectorios necesarios para guardar los datos de manera ordenada. Tambien es importante evitar cargar datos que ya han sido descargados anteriormente.

In [ ]:
# pwd
print(os.getcwd())

#  Estos son los datos que vamos a descargar y donde vamos a guardarlos
desaparecidos_RNPDNO_url = "http://www.datamx.io/dataset/fdd2ca20-ee70-4a31-9bdf-823f3c1307a2/resource/d352810c-a22e-4d72-bb3b-33c742c799dd/download/desaparecidos3ago.zip"
desaparecidos_RNPDNO_archivo = "desaparecidosRNPDNO.zip"
desaparecidos_corte_nacional_url = "http://www.datamx.io/dataset/fdd2ca20-ee70-4a31-9bdf-823f3c1307a2/resource/4865e244-cf59-4d39-b863-96ed7f45cc70/download/nacional.json"
desaparecidos_corte_nacional_archivo = "desaparecidos_nacional.json"
subdir = "./data/"


In [ ]:
if not os.path.exists(desaparecidos_RNPDNO_archivo):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    urllib.request.urlretrieve(desaparecidos_RNPDNO_url, subdir + desaparecidos_RNPDNO_archivo)  
    with zipfile.ZipFile(subdir + desaparecidos_RNPDNO_archivo, "r") as zip_ref:
        zip_ref.extractall(subdir)
    
    urllib.request.urlretrieve(desaparecidos_corte_nacional_url, subdir + desaparecidos_corte_nacional_archivo)  

    with open(subdir + "info.txt", 'w') as f:
        f.write("Archivos sobre personas desaparecidas\n")
        info = """
        Datos de desaparecidos, corte nacional y desagregación a nivel estatal, 
        por edad, por sexo, por nacionalidad, por año de desaparición y por mes
        de desaparición para los últimos 12 meses.

        Los datos se obtuvieron del RNPDNO con fecha de 03 de agosto de 2021
        (la base de datos no se ha actualizado últimamente) 

        """ 
        f.write(info + '\n')
        f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write("Desde: " + desaparecidos_RNPDNO_url + "\n")
        f.write("Nombre: " + desaparecidos_RNPDNO_archivo + "\n")
        f.write("Agregados nacionales descargados desde: " + desaparecidos_corte_nacional_url + "\n")
        f.write("Nombre: " + desaparecidos_corte_nacional_archivo + "\n")

# 2. Archivos en formato `json`

Los archivos en formato json son posiblemente los más utilizados actualmente para transferir información por internet, ya que se usa en prácticamente todas las REST API. Como acabamos de ver es normal tener que enfrentarse con archivos `json` pésimamente o nada documentados, por lo que es necesario saber como tratarlos. 

Vamos a ver como se hace eso utilizando la bibloteca de `json`y la de `pandas`. Para `pandas`les recomiendo, si no lo conocen, de darle una vuelta a [la documentación y los tutoriales](https://pandas.pydata.org/docs/) que está muy bien hecha. O a el [curso básico de Kaggle](https://www.kaggle.com/learn/pandas).

Sobre `json`, posiblemente [la página con la especificación](https://www.json.org/json-en.html) sea más que suficiente. 

Vamos a hacer un ejemplito sencillo y carismático revisando los repositorios de [github](https://github.com) y les voy a dejar que exploren los `json` de los archivos de personas desaparecidas.

In [ ]:
import pandas as pd # Esto es como una segunda piel
import json # Una forma estandar de leer archivos json 

archivo_url = "https://api.github.com/users/google/repos"
archivo_nombre = "repos-google.json"
subdir = "./data/"

if not os.path.exists(subdir + archivo_nombre):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    urllib.request.urlretrieve(archivo_url, subdir + archivo_nombre)


Vamos primero a ver como le hacemos con `pandas`

In [ ]:
df_repos = pd.read_json(subdir + archivo_nombre)

df_repos.head()

In [ ]:
df_repos.info()

y ahora como le hacemos con la biblioteca de `json`

In [ ]:
with open(subdir + archivo_nombre, 'r') as fp:
    repos = json.load(fp)

print(f"\nNúmero de entradas: {len(repos)}")
print(f"\nNombre de los atributos: { ", ".join(repos[0].keys())}")
print(f"\nAtributos de 'owner': {", ".join(repos[0]['owner'].keys())}")


### Ejercicio

Utiliza los archivos `json` descargados con el detalle a nivel estatal, y genera unos 3 `DataFrame` con información sobre personas desaparecidas dependiendo de diferentes características. 

# 3. Archivos xml

Los archivos *xml* son una manera de compartir información a través de internet o de guardar información con formatos genéricos que sigue siendo muy utilizada hoy en día. En general lidiar con archivos xml es una pesadilla y se necesita explorarlos con calma y revisarlos bien antes de usarlos. 

La definición del formato y su uso se puede revisar en [este tutorial de la w3schools](https://www.w3schools.com/xml/default.asp). Vamos a ver un ejemplo sencillo basado en la librería [xml.etree.ElementTree](https://docs.python.org/3/library/xml.etree.elementtree.html) que viene de base en python:


In [ ]:
import xml.etree.ElementTree as et 

archivo_url = "https://github.com/mcd-unison/ing-caract/raw/main/ejemplos/integracion/ejemplos/ejemplo.xml"
archivo_nombre = "ejemplito.xml"
subdir = "./data/"

if not os.path.exists(subdir + archivo_nombre):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    urllib.request.urlretrieve(archivo_url, subdir + archivo_nombre)


desayunos = et.parse(subdir + archivo_nombre)

for (i, des) in enumerate(desayunos.getroot()):
    print("Opción {}:".format(i+1))
    for prop in des:
        print("\t{}: {}".format(prop.tag, prop.text.strip()))

# Se puede buscar por etiquetas y subetiquetas

print("Los desayunos disponibles son: " + 
      ", ".join([p.text for p in desayunos.findall("food/name")]))

# ¿Como se podría poner esta información en un DataFrame de `pandas`?
# Agreguen tanto código como consideren necesario.


Wikipedia es un buen ejemplo de un lugar donde la información se guarda y se descarga en forma de archivos xml. Por ejemplo, si queremos descargar datos de la wikipedia [con su herramienta de exportación](https://es.wikipedia.org/wiki/Especial:Exportar) utilizando [las categorias definidas por Wikipedia](https://es.wikipedia.org/wiki/Portal:Portada). Para hacerlo en forma programática es ecesario usar la [API de Mediawiki](https://github.com/mudroljub/wikipedia-api-docs) que veremos más adelante.

Por el momento descargemos unos datos de *wikipedia* y hagamos el ejercicio de tratar de entender la estructura del árbol.

In [ ]:
archivo_url = "https://github.com/mcd-unison/ing-caract/raw/main/ejemplos/integracion/ejemplos/wikipedia-poetas.xml"
archivo_nombre = "poetas.xml"
subdir = "./data/"

if not os.path.exists(subdir + archivo_nombre):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    urllib.request.urlretrieve(archivo_url, subdir + archivo_nombre)


poetas = et.parse(subdir + archivo_nombre)


### Ejercicio

Entender la estructura del archivo `xml` de poetas y generar un `DataFrame` con la información más importante. No olvides de comentar tu código y explicar la estructura del archivo `xml`

# 4. Archivos de Excel

Los archivos de excel son a veces nuestros mejores amigos, y otras veces nuestras peores pesadillas. Un archivo en excel (o cualquier otra hoja de caálculo) son formatos muy útiles que permiten compartir información técnica con personas sin preparación técnica, lo que lo vuelve una herramienta muy poderosa para comunicar hallazgos a los usuarios.

Igualmente, la manipulación de datos a través de hojas de cálculo, sin usarlas correctamente 8esto es, programando cualquier modificación) genera normalmente un caos y una fuga de información importante para una posterior toma de desición. 

Como buena práctica, si se tiene acceso a la fuente primaria de datos y se puede uno evitar el uso de datos procesados en hoja de calculo, siempre es mejor esa alternativa (como científico de datos o analista de datos). Pero eso muchas veces es imposible.

Vamos a dejar la importación desde `xlsx` a los cursos de *DataCamp* que lo tratan magistralmente. Es importante que, para que se pueda importar desde python o R, muchas veces es necesario instalar librerías extras.